<a href="https://colab.research.google.com/github/Manisha2297/RestroRecommender/blob/main/NLTKmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
import string

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#mount your own drive to avoid downloading the data multiple time 

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered.csv')
df_review.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# df_review = pd.read_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/yelp_academic_dataset_review.csv')
# df_review.drop('Unnamed: 0', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# restaurant_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/restuarant_df')

In [7]:
# business_id_list = list(restaurant_df.business_id.values)

In [8]:
# df_review_filtered = df_review[df_review['business_id'].isin(business_id_list)]

In [14]:
# pd.DataFrame(df_review_filtered.user_id.unique(), columns=['user_id']).to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/unique_users_df')

In [26]:
# df_review_filtered.to_csv('/content/gdrive/MyDrive/YelpDataset/CsvFiles/review_filtered.csv')

In [5]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,HQl28KMwrEKHqhFrrDqVNQ,0,2015-12-05 03:18:11,0.0,LG2ZaYiOgpr2DK_90pYjNw,5.0,I love Deagan's. I do. I really do. The atmosp...,1.0,V34qejxNsCbcgD8C0HVk-Q
1,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0.0,6TdNDKywdbjoTkizeMce8A,4.0,"Oh happy day, finally have a Canes near my cas...",0.0,UgMW8bLE0QMJDCkQ1Ax5Mg
2,nlxHRv1zXGT0c0K51q3jDg,0,2013-05-07 07:25:25,0.0,L2O_INwlrRuoX05KSjc4eg,5.0,This is definitely my favorite fast food sub s...,2.0,5vD2kmE25YBrbayKhykNxQ
3,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0.0,ZayJ1zWyWgY9S_TRLT_y9Q,5.0,"Really good place with simple decor, amazing f...",1.0,aq_ZxGHiri48TUXJlpRkCQ
4,e_BiI4ej1CW1F0EyVLr-FQ,0,2015-02-16 06:48:47,0.0,JA-xnyHytKiOIHl_ztnK9Q,5.0,Most delicious authentic Italian I've had in t...,0.0,P6apihD4ASf1vpPxHODxAQ


In [6]:
#Select only stars and text
yelp_data = df_review[['business_id', 'user_id', 'stars', 'text']]

In [7]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [8]:
yelp_data['text'].fillna("", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [9]:
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they have any punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

In [10]:
# size = 0
# processed_text_df = pd.DataFrame()
# for i in range(2000000, len(yelp_data), 10000):
#   processed_text_df = pd.concat([processed_text_df, yelp_data['text'][i:i+10000].apply(text_process)])
#   size+=10000

#   if len(processed_text_df) >= size:
#     if len(processed_text_df)%100000==0:
#       print(len(processed_text_df))
#     processed_text_df.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
#     del processed_text_df
#     processed_text_df = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_2')
  
#   else:
#     print(size)
#     break


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2610000


In [11]:
len(yelp_data)

4607291

In [12]:
processed_text_df_1 = pd.read_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df_1')

In [15]:
processed_text_df_final = pd.concat([processed_text_df_1, processed_text_df])

In [17]:
processed_text_df_final.to_pickle('/content/gdrive/MyDrive/YelpDataset/PickleFiles/processed_text_df')

In [19]:
yelp_data['text'][0]

"I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sauce they come with is to die for. The beer list is amazing and the cocktails are great. The prices are mid-level, so it's not a cheap dive you can go to every week, but rather a treat when you do. Try it out. You won't be disappointed!"

In [24]:
processed_text_df_final.head()

,0
0,love Deagans really atmosphere cozy festive sh...
1,Oh happy day finally Canes near casa Yes other...
2,definitely favorite fast food sub shop Ingredi...
3,Really good place simple decor amazing food gr...
4,delicious authentic Italian Ive US leave place...


In [ ]:
yelp_data['text'] = 

In [18]:
del processed_text_df_1
del processed_text_df

In [ ]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [ ]:
yelp_data['text']

In [ ]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])

In [ ]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])

In [ ]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])

In [ ]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


In [ ]:
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

In [ ]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [ ]:
yelp_data.to_pickle('/content/gdrive/MyDrive/Yelp Dataset/yelp_review_df')
del yelp_data

In [ ]:
#Loading in the dataframe
yelp_data = pd.read_pickle('/content/gdrive/MyDrive/Yelp Dataset/yelp_review_df')
yelp_data.head()